- 웹드라이버/라이브러리 불러오기

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pyautogui as pg
import pandas as pd

In [ ]:
t = 115 #'총교육시간(계산용)'
p = 14000 #'시간당 표준단가(계산용)'
cn = '22-003차 포스코그룹 신입사원 입문교육'

- 러닝플랫폼 접속

In [ ]:
# 계정입력 
lp_id = "user@poscohrd.com"
lp_pw = pg.password(title="lp패스워드", text="패스워드 입력하기")

#러닝플랫폼 접속
url = 'https://lp.posco.co.kr/S22/S22A10/member/login/loginForm.do'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
driver.maximize_window()

# 접속후, 로그인
driver.find_element(By.XPATH,'//*[@id="email"]').send_keys(lp_id)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(lp_pw)
driver.find_element(By.XPATH,'//*[@id="loginForm"]/div[2]/button').click()

# 로그인 이후, 대기시간 지정
pg.sleep(3)

# 홍보 팝업창 off / 바뀜
driver.find_element(By.XPATH,'//*[@id="div_4326"]/div/div[2]/button[1]').click()

# 로그인 이후, 대기시간 지정
pg.sleep(3)


교육담당자모드 선택

In [ ]:
#프로필에서 교육담당자 모드 선택
driver.find_element(By.XPATH,'//*[@id="btnMy"]').click()
driver.find_element(By.XPATH,'/html/body/header/div[2]/div[4]/select/option[1]').click()
driver.find_element(By.XPATH,'/html/body/header/div[2]/div[4]/select/option[2]').click()

In [ ]:
# [과정관리]-[과정차수]로 이동
url = 'https://lp.posco.co.kr/S22/S22A10/admin/course/courseCs.do'
driver.get(url)

# [교육기간]-[전체] / [담당자]-[전체] 선택
xp = '//*[@id="searchForm"]/div/ul/li/input[1]'
driver.find_element(By.XPATH,xp).click()

xp = '//*[@id="MyManager"]/option[1]'
driver.find_element(By.XPATH,xp).click()

# 찾고자 하는 [교육차수] 검색
xp = '//*[@id="keyword"]'
driver.find_element(By.XPATH,xp).click()

# 찾고자 하는 [교육차수명] 입력
xp = '//*[@id="btnView1"]'
search_key = cn # 상단에 [코스네임으로 변수 저장]
driver.find_element(By.XPATH,'//*[@id="keyword"]').send_keys(search_key)
driver.find_element(By.XPATH,xp).click()

pg.sleep(3)

# 찾은 [교육차수명] 선택
xp = '//*[@id="grid"]/tbody/tr/td[3]/a'
driver.find_element(By.XPATH,xp).click()

# [운영관리] 이동
xp = '//*[@id="btnStudentMgnt"]'
driver.find_element(By.XPATH,xp).click()

# [2,000개씩 보기] 이동
xp = '//*[@id="grid_length"]/label/select/option[7]'
driver.find_element(By.XPATH,xp).click()

# [엑셀 다운로드] 이동
xp = '//*[@id="btnExlDwl1"]'
driver.find_element(By.XPATH,xp).click()

In [ ]:
# 교육날짜 쓰기
# 교육시작일
xp = '//*[@id="eduStartDt"]' 
sdt = driver.find_element(By.XPATH,xp).text
# 교육종료일
xp = '//*[@id="eduEndDt"]' 
edt =driver.find_element(By.XPATH,xp).text


In [ ]:
# [엑셀 열기] 이동
df = pd.read_excel("D:\Downloads\입과자 리스트.xls")
# df.head(10) # 헤드로부터 10행까지 한다면, 확인
df

In [ ]:
# 해당 열만 선택(열은 0-기준 / 행은 unname 기준)
df.loc[0]
df2 = df.drop(1) #drop은 삭제 #모르면 df, df2, df3으로 확인해보기
df3 = df2.rename(columns=df.loc[0])
df4 = df3.drop(0)
df5 = df4[['이름','직번','회사']]
df5

In [ ]:
cc = pd.read_excel('회사명_코드_리스트.xlsx')
cc

In [ ]:
member = pd.merge(df5,cc,how='left',on='회사')
member

In [ ]:
member.drop('Unnamed: 0',1,inplace=True) #drop은 삭제 #모르면 df, df2, df3으로 확인해보기

In [ ]:
idx = member[(member["c_code"].isnull())].index
member.drop(idx,inplace=True) #inplace는 변경된 내용을 원본 데이터에 반영
member

In [ ]:
member['c_code']=member['c_code'].astype('int')
member

In [ ]:
member['월'] = 12
member['활동센터'] = 'C221064_포스코그룹 경력사원 입문교육'
member['분류'] = '사외위탁'
member['유형'] = '위탁'
member['필수/구분'] = '필수'
member['교육과정명_통합'] = '포스코그룹 경력사원 입문교육'
member['교육과정명'] = cn
member['교육기간From'] = sdt
member['교육기간To'] = edt
member['총교육시간'] = 115 #t
member['시간당 표준단가'] = 14000 #p
member['수료여부'] = '수료'
member['고용보험 환급여부'] = 'X'
member['교육과정 中 특이사항'] = ' '
member['운영부서'] = '리더십교육그룹'
member['운영담당자'] = '공승규'
member['소속해외법인'] = ' '
member['해외법인부담(Usd)'] = ' '
member['해외법인부담(Krw)'] = 0
member['교육비'] = t*p
member['교육비해외법인'] = t*p-0 #엑셀수식 참고

In [ ]:
member2 = member[['월','구분','활동센터','분류','유형','필수/구분','교육과정명_통합','교육과정명','교육기간From','교육기간To','시간당 표준단가','c_code','company','이름','직번','수료여부','고용보험 환급여부','교육과정 中 특이사항','운영부서','운영담당자','소속해외법인','해외법인부담(Usd)','해외법인부담(Krw)','교육비','교육비해외법인']]
member2.to_excel('11월 교육수료현황.xlsx')